# Exercise

Problem description: https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant
- Data is provided in two formats - xlsx and ods. Use pd.read_excel to load the data .xlsx file. The excel file has 5 sheets. Create a dataframe for each and concatenate them into a single dataframe. [Hint: use pd.concat function]. Note: one column need to renamed: PE => EP to be consistent with the problem statement.
- Create training and test set with 70/30 ratio and random seed = 1. Predict EP based on the other variables as features (AT, RH, V and AP). 
- Calculate R2 and RMSE for training and test data. [Answer: 0.9287, 0.9284 (r2) 4.55 4.57 (rmse)]
- Find the residuals (ypredict - ytrue) on the test data and plot histogram to see its distribution. Ideally the histogram of the residuals should look "gaussian normal". Do a scatter plot for residual vs actual. Observe whether the residuals are consistently same for entire range of actual. 
- Which features are positively related with the outcome and which are negatively related?
- Which feature is the strongest predictor?
- Improve your model using log transformation of the output and polynomial transformation of the features with degree = 2 [Answer: 0.9371, 0.9369]. Also, plot the residual on the test data.
